In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "3_clean")

# Decision Trees and Random Forest Models

In [2]:
df= pd.read_csv(os.join(inp, "analysis_bbl.csv")).drop("Unnamed: 0", axis = 1)
df.head()

,borough,cd,ct2010,zipcode,address,bldgclass,landuse,ownertype,ownername,lotarea,...,holc_D,num_vac_orders,vacate_ind,num_vacate_orders,viol_count,viol_count_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
0,MN,102.0,61.0,10003.0,42 EAST 12 STREET,R1,2.0,NaN,UNAVAILABLE OWNER,2643.0,...,0.0,NaN,0,0.0,1.0,0.166667,0.0,0.0,0.0,1.0
1,BK,315.0,392.0,11223.0,2105 OCEAN PARKWAY,A9,1.0,NaN,OCEAN PARKWAY BH 26 LLC,3445.0,...,0.0,NaN,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
2,BK,315.0,388.0,11223.0,2287 EAST 1 STREET,B3,1.0,NaN,2287E1 LLC,1500.0,...,1.0,NaN,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
3,QN,408.0,450.0,11432.0,161-26 GRAND CENTRAL PKWY,A5,1.0,NaN,CARLOS RODRIGUEZ,2925.0,...,0.0,NaN,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
4,QN,413.0,1617.0,11426.0,244-14 91 AVENUE,A9,1.0,NaN,BASHIR IRFAN,6086.0,...,0.0,NaN,0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0


In [3]:
print(df.columns)
df.describe()

Index(['borough', 'cd', 'ct2010', 'zipcode', 'address', 'bldgclass', 'landuse',
       'ownertype', 'ownername', 'lotarea', 'bldgarea', 'numbldgs',
       'numfloors', 'unitsres', 'unitstotal', 'assessland', 'assesstot',
       'exempttot', 'yearbuilt', 'yearalter1', 'yearalter2', 'bbl', 'sanborn',
       'holc_AB', 'holc_CD', 'holc_D', 'num_vac_orders', 'vacate_ind',
       'num_vacate_orders', 'viol_count', 'viol_count_per_unit', 'AL', 'NB',
       'average_energy_usage', 'average_energy_usage_mi'],
      dtype='object')


,cd,ct2010,zipcode,landuse,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,...,holc_D,num_vac_orders,vacate_ind,num_vacate_orders,viol_count,viol_count_per_unit,AL,NB,average_energy_usage,average_energy_usage_mi
count,765323.000000,765264.000000,765316.000000,765714.000000,7.654280e+05,7.657140e+05,765428.000000,765428.000000,765708.000000,765712.000000,...,765117.00000,2603.000000,765714.000000,763111.0,765714.000000,763004.0,765714.000000,765714.000000,765389.000000,765714.000000
mean,357.693954,470.832974,11026.161084,1.423121,3.815063e+03,5.078128e+03,1.324699,2.447578,4.717486,4.981860,...,0.22371,1.094891,0.003399,0.0,7.081562,inf,0.002844,0.000876,4.421240,0.976662
std,98.110120,378.922236,463.572291,0.844367,2.317468e+04,3.521141e+04,2.768148,1.693242,42.120376,63.182793,...,0.41673,0.438168,0.058206,0.0,50.126042,NaN,0.026303,0.010436,850.251643,0.150974
min,101.000000,1.000000,10001.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.00000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,307.000000,170.090000,10467.000000,1.000000,2.000000e+03,1.442000e+03,1.000000,2.000000,1.000000,1.000000,...,0.00000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
50%,403.000000,363.000000,11223.000000,1.000000,2.500000e+03,2.023000e+03,1.000000,2.000000,2.000000,2.000000,...,0.00000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
75%,412.000000,671.000000,11372.000000,2.000000,4.000000e+03,2.928000e+03,2.000000,2.750000,2.000000,2.000000,...,0.00000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
max,503.000000,1621.000000,12345.000000,4.000000,1.225343e+07,1.354011e+07,1854.000000,98.000000,19201.000000,36481.000000,...,1.00000,15.000000,1.000000,0.0,4825.000000,inf,13.444444,1.333333,400064.000000,1.000000


In [4]:
df= pd.get_dummies(df, columns=["borough"])
borocols= ["borough_" + x for x in ["BK", "BX", "MN", "QN", "SI"]]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X = df.loc[:, ["landuse", "lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
              "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "viol_count", "AL", "NB",
              "average_energy_usage", "average_energy_usage_mi"]+borocols].fillna(0)
y = df.loc[:, "vacate_ind"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

dt = DecisionTreeClassifier(max_depth=10)
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
accuracy= (y_predict == y_test).value_counts(normalize=True)[True]
accuracy

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Jennah\AppData\Local\Temp/ipykernel_21668/82528116.py", line 11, in <module>
    dt.fit(X_train, y_train)
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\sklearn\tree\_classes.py", line 420, in fit
    builder.build(self.tree_, X, y, sample_weight)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During ha

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Jennah\AppData\Local\Temp/ipykernel_21668/82528116.py", line 11, in <module>
    dt.fit(X_train, y_train)
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\sklearn\tree\_classes.py", line 420, in fit
    builder.build(self.tree_, X, y, sample_weight)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During ha

__Area under the curve__

In [5]:
from sklearn.metrics import roc_auc_score
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
# predict_proba predicts the probability of each class rather than just the most likely class
pred=dt.predict_proba(X_test)[:,1] # predicted probability of y = 1
roc_auc_score(np.array(y_test),pred)

ERROR! Session/line number was not unique in database. History logging moved to new session 231


KeyboardInterrupt: 

In [ ]:
# vs. random chance (only slightly better)
pred=np.random.random(len(X_test))
roc_auc_score(np.array(y_test.apply(int)),pred)

## Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
# This time we'll use max_depth to control the complexity of the tree, still using the same train/test split as above,
# and optimize the parameter value using GridSearchCV.
param_grid = {'max_depth':range(1,20)}
dt=DecisionTreeClassifier()
gr=GridSearchCV(dt,param_grid=param_grid,scoring='roc_auc')
rs=gr.fit(X_train,y_train)
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),rs.predict_proba(X_test)[:,1]))

## Feature Importance

In [ ]:
dt = DecisionTreeClassifier(max_depth=rs.best_params_['max_depth'])
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]

## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_leaf_nodes=10)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,pred))

In [ ]:
param_grid = {'n_estimators':range(1,51,5)}
rf = RandomForestClassifier(n_jobs=-1,max_leaf_nodes=10)
gs = GridSearchCV(rf,param_grid=param_grid,scoring='roc_auc')
rs = gs.fit(X_train,y_train)
pred=rs.predict_proba(X_test)[:,1]
print(rs.best_params_)
print(roc_auc_score(np.array(y_test),pred))

# Naive Bayes?